# Chicago Crime Data 
* Jacob Wang
* Coding Dojo Data Science Project 4 - Part 1 
* 2/4/2023

## Task
* Your task is to answer a series of questions about trends in crimes in Chicago for a reporter for the local newspaper.

#### **Questions:** 

1) Comparing Police Districts:
    * Which district has the most crimes? Which has the least?
    
    
2) Crimes Across the Years:
    * Is the total number of crimes increasing or decreasing across the years?
    * Are there any individual crimes that are doing the opposite (e.g decreasing when overall crime is increasing or vice-versa)?
    
    
3) Comparing Months:
    * What months have the most crime? What months have the least?
    

### Imports/Load data

In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticks
import seaborn as sns


import missingno as miss
import datetime as dt
import statsmodels.tsa.api as tsa

In [2]:
folder = "Data/Chicago/"
crime_files = sorted(glob.glob(folder+"*.csv"))
df = pd.concat([pd.read_csv(f) for f in crime_files])

In [8]:
df.info()
df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7727169 entries, 0 to 17811
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int64  
 1   Date                  object 
 2   Primary Type          object 
 3   Description           object 
 4   Location Description  object 
 5   Arrest                bool   
 6   Domestic              bool   
 7   Beat                  int64  
 8   District              float64
 9   Ward                  float64
 10  Latitude              float64
 11  Longitude             float64
dtypes: bool(2), float64(4), int64(2), object(4)
memory usage: 663.2+ MB


,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Latitude,Longitude
0,3206463,01/01/2001 01:00:00 AM,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,True,835,8.0,18.0,41.735507,-87.690095
1,1358218,01/01/2001 01:00:00 PM,ASSAULT,SIMPLE,APARTMENT,False,False,312,3.0,NaN,41.778744,-87.612135
2,1323691,01/01/2001 01:00:00 AM,WEAPONS VIOLATION,UNLAWFUL POSS OF HANDGUN,STREET,False,False,1135,11.0,NaN,41.866694,-87.688513
3,1324743,01/01/2001 01:00:00 PM,GAMBLING,ILLEGAL ILL LOTTERY,STREET,True,False,313,3.0,NaN,41.780412,-87.611970
4,1317937,01/01/2001 01:00:00 AM,BATTERY,SIMPLE,RESIDENCE PORCH/HALLWAY,False,False,334,3.0,NaN,41.762514,-87.565877
...,...,...,...,...,...,...,...,...,...,...,...,...
17807,12965291,01/27/2023 12:48:00 PM,CONCEALED CARRY LICENSE VIOLATION,ARMED WHILE UNDER THE INFLUENCE,STREET,True,False,1821,18.0,27.0,41.909828,-87.638553
17808,12969720,01/27/2023 12:49:00 PM,THEFT,FROM BUILDING,APARTMENT,False,False,2521,25.0,31.0,41.935410,-87.756571
17809,12964887,01/27/2023 12:49:00 AM,OTHER OFFENSE,OBSCENE TELEPHONE CALLS,POLICE FACILITY / VEHICLE PARKING LOT,False,False,1225,12.0,28.0,41.872333,-87.679608
17810,12965449,01/27/2023 12:50:00 PM,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,STREET,False,False,1914,19.0,46.0,41.966244,-87.658013


## 1.) Comparing Police Districts:
* Which district has the most crimes? Which has the least?

#### Set datetime as index

In [11]:
date_format = "%m/%d/%Y %H:%M:%S %p"
df['Datetime'] = pd.to_datetime(df['Date'], format=date_format)
df = df.sort_values('Datetime')
df = df.set_index('Datetime')
df.head(3)

,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Latitude,Longitude
Datetime,,,,,,,,,,,,
2001-01-01 01:00:00,3206463,01/01/2001 01:00:00 AM,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,True,835,8.0,18.0,41.735507,-87.690095
2001-01-01 01:00:00,1311503,01/01/2001 01:00:00 AM,CRIM SEXUAL ASSAULT,AGGRAVATED: OTHER DANG WEAPON,BAR OR TAVERN,False,False,1933,19.0,NaN,41.931374,-87.648819
2001-01-01 01:00:00,6808288,01/01/2001 01:00:00 PM,THEFT,FINANCIAL ID THEFT: OVER $300,APARTMENT,False,False,213,2.0,3.0,41.822551,-87.615632


In [13]:
df_district = df.groupby('District')
